In [1]:
# %%capture

# !pip install obonet

In [1]:
import pickle
import ujson
import sys
import os

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import defaultdict
from typing import Optional

from bigbio.dataloader import BigBioConfigHelpers

sys.path.append('../../../..')
from umls_utils import UmlsMappings
from bigbio_utils import CUIS_TO_REMAP, CUIS_TO_EXCLUDE, DATASET_NAMES, VALIDATION_DOCUMENT_IDS
from bigbio_utils import dataset_to_documents, dataset_to_df, resolve_abbreviation, get_left_context, get_right_context

conhelps = BigBioConfigHelpers()

/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for an_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/an_em/an_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:923: FutureWarning: The repository for anat_em contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /home2/cye73/biomedical-entity-linking/biomedical/bigbio/hub/hub_repos/anat_em/anat_em.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True`

In [2]:
# %%capture
# ! pip install datasets

In [ ]:
# %%capture
# ! pip install ujson

In [ ]:
# print(conhelps.available_dataset_names)
# for helper in conhelps:
#     print(helper.config.name)

# Personal Tests

In [5]:
name = 'bc5cdr'
data = conhelps.for_config_name(f'{name}_bigbio_kb').load_dataset()
exclude = CUIS_TO_EXCLUDE[name]
remap = CUIS_TO_REMAP[name]


df = dataset_to_df(data, entity_remapping_dict=remap, cuis_to_exclude=exclude)
docs = dataset_to_documents(data)

df[10:20]

/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/bc5cdr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/bc5cdr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


,document_id,offsets,text,type,db_ids,split,mention_id
27,10027919,"[[398, 414]]",22-oxacalcitriol,[Chemical],[MESH:C051883],validation,10027919.11
28,10027919,"[[416, 419]]",OCT,[Chemical],[MESH:C051883],validation,10027919.12
29,10027919,"[[526, 529]]",OCT,[Chemical],[MESH:C051883],validation,10027919.13
30,10027919,"[[591, 614]]",impaired renal function,[Disease],[MESH:D007674],validation,10027919.14
31,10027919,"[[743, 752]]",phosphate,[Chemical],[MESH:D010710],validation,10027919.15
33,10027919,"[[813, 822]]",phosphate,[Chemical],[MESH:D010710],validation,10027919.16
34,10027919,"[[887, 890]]",OCT,[Chemical],[MESH:C051883],validation,10027919.17
2,10027919,"[[1110, 1117]]",calcium,[Chemical],[MESH:D002118],validation,10027919.18
3,10027919,"[[1230, 1233]]",OCT,[Chemical],[MESH:C051883],validation,10027919.19
4,10027919,"[[1317, 1320]]",OCT,[Chemical],[MESH:C051883],validation,10027919.20


In [2]:
name = 'medmentions_st21pv'
data = conhelps.for_config_name(f'{name}_bigbio_kb').load_dataset()
exclude = CUIS_TO_EXCLUDE[name]
remap = CUIS_TO_REMAP[name]


df = dataset_to_df(data, entity_remapping_dict=remap, cuis_to_exclude=exclude)
docs = dataset_to_documents(data)


df['label_len'] = df['db_ids'].map(lambda x: len(x))
df[df.label_len == df.label_len.max()]


/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/medmentions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/medmentions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


,document_id,offsets,text,type,db_ids,split,mention_id,label_len
64613,27655319,"[[462, 468]]",detect,"[T033, T058]","[UMLS:C0442726, UMLS:C1511790]",test,27655319.17,2
72785,27708774,"[[2036, 2052]]",antral follicles,[T017],"[UMLS:C1135971, UMLS:C0600225]",train,27708774.75,2
87101,27801847,"[[496, 500]]",Ldlr,"[T017, T103]","[UMLS:C0034821, UMLS:C1366529]",train,27801847.22,2
111221,27979025,"[[100, 104]]",food,"[T038, T168]","[UMLS:C0016470, UMLS:C0016452]",train,27979025.4,2
189426,28464073,"[[2110, 2112]]",HS,"[T038, T037]","[UMLS:C0282507, UMLS:C0282498]",train,28464073.47,2
199085,28526565,"[[1372, 1377]]",ICOAS,"[T058, T201]","[UMLS:C1456627, UMLS:C0087111]",train,28526565.38,2
202565,28548949,"[[1809, 1812]]",XPA,"[T017, T103]","[UMLS:C1337030, UMLS:C1506534]",train,28548949.75,2


In [4]:
name = 'ncbi_disease'
data = conhelps.for_config_name(f'{name}_bigbio_kb').load_dataset()
exclude = CUIS_TO_EXCLUDE[name]
remap = CUIS_TO_REMAP[name]


df = dataset_to_df(data, entity_remapping_dict=remap, cuis_to_exclude=exclude)
docs = dataset_to_documents(data)

display(df)


/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/ncbi_disease
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


,document_id,offsets,text,type,db_ids,split,mention_id
2,10021369,"[[43, 76]]",adenomatous polyposis coli tumour,[Modifier],[MESH:D011125],train,10021369.1
3,10021369,"[[93, 132]]",adenomatous polyposis coli (APC) tumour,[Modifier],[MESH:D011125],train,10021369.2
1,10021369,"[[357, 372]]",colon carcinoma,[Modifier],[MESH:D003110],train,10021369.3
4,10021369,"[[955, 970]]",colon carcinoma,[Modifier],[MESH:D003110],train,10021369.4
0,10021369,"[[1090, 1096]]",cancer,[SpecificDisease],[MESH:D009369],train,10021369.5
...,...,...,...,...,...,...,...
6804,9988281,"[[996, 1015]]",breast malignancies,[SpecificDisease],[MESH:D001943],test,9988281.7
6794,9988281,"[[1123, 1147]]",invasive lobular cancers,[DiseaseClass],[MESH:D018275],test,9988281.8
6795,9988281,"[[1152, 1179]]",low-grade ductal carcinomas,[SpecificDisease],[MESH:D044584],test,9988281.9
6797,9988281,"[[1269, 1286]]",ductal carcinomas,[SpecificDisease],[MESH:D044584],test,9988281.10


In [7]:
name2 = 'bc5cdr'
data2 = conhelps.for_config_name(f'{name2}_bigbio_kb').load_dataset()
exclude = CUIS_TO_EXCLUDE[name2]
remap = CUIS_TO_REMAP[name2]


df2 = dataset_to_df(data2, entity_remapping_dict=remap, cuis_to_exclude=exclude)
docs2 = dataset_to_documents(data2)


df2['label_len'] = df2['db_ids'].map(lambda x: len(x))


# df2[df2.label_len == df2.label_len.max()]
print("\n\ndata2 :\n\n", data2)
print("\n\ndf2 :\n\n")
display(df2)


/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/bc5cdr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/bc5cdr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(




data2 :

 DatasetDict({
    train: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
        num_rows: 500
    })
})


df2 :




,document_id,offsets,text,type,db_ids,split,mention_id,label_len
0,10027919,"[[0, 16]]",22-oxacalcitriol,[Chemical],[MESH:C051883],validation,10027919.1,1
23,10027919,"[[28, 57]]",secondary hyperparathyroidism,[Disease],[MESH:D006962],validation,10027919.2,1
32,10027919,"[[75, 92]]",low bone turnover,[Disease],[MESH:D001851],validation,10027919.3,1
1,10027919,"[[106, 119]]",renal failure,[Disease],[MESH:D051437],validation,10027919.4,1
5,10027919,"[[133, 143]]",Calcitriol,[Chemical],[MESH:D002117],validation,10027919.5,1
...,...,...,...,...,...,...,...,...
28802,9952311,"[[1145, 1156]]",bradycardia,[Disease],[MESH:D001919],validation,9952311.13,1
28803,9952311,"[[1332, 1340]]",swelling,[Disease],[MESH:D004487],validation,9952311.14,1
28804,9952311,"[[1612, 1615]]",ADR,[Chemical],[MESH:D004317],validation,9952311.15,1
28807,9952311,"[[1799, 1802]]",ADR,[Chemical],[MESH:D004317],validation,9952311.16,1


In [8]:
filtered_df2 = df2[df2['document_id'] == "227508"]
print(filtered_df2)


      document_id         offsets              text        type  \
16045      227508        [[0, 8]]          Naloxone  [Chemical]   
16055      227508      [[49, 58]]         clonidine  [Chemical]   
16061      227508     [[93, 105]]      hypertensive   [Disease]   
16048      227508    [[181, 190]]         clonidine  [Chemical]   
16049      227508    [[274, 285]]       hypotensive   [Disease]   
16050      227508    [[306, 322]]  alpha-methyldopa  [Chemical]   
16051      227508    [[354, 362]]          naloxone  [Chemical]   
16052      227508    [[364, 372]]          Naloxone  [Chemical]   
16053      227508    [[469, 481]]      hypertensive   [Disease]   
16054      227508    [[487, 496]]         clonidine  [Chemical]   
16056      227508    [[589, 597]]          naloxone  [Chemical]   
16057      227508    [[637, 646]]         clonidine  [Chemical]   
16058      227508    [[750, 762]]      hypertensive   [Disease]   
16059      227508    [[865, 873]]          naloxone  [Chemical

In [9]:
data2_doc = dataset_to_documents(data2)
print(data2_doc['227508'])
print(len(data2_doc['227508']))


Naloxone reverses the antihypertensive effect of clonidine.
In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release o

In [10]:
left_context = get_left_context(data2_doc['227508'], start = 500)
print(left_context)

In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10


In [11]:
import os
print(os.getcwd())

/home2/cye73/biomedical-entity-linking/MyTests


In [14]:
entity_dict = pickle.load(open('/home2/cye73/data/arboel/medmentions_st21pv/dictionary.pickle', 'rb'))
entity_dict[:5] # Print the first 10 keys to check their format

[{'cui': 'UMLS:C0000005',
  'title': '(131)I-Macroaggregated Albumin',
  'type': 'T116',
  'description': '(131)I-Macroaggregated Albumin ( T116 : (131)I-MAA )'},
 {'cui': 'UMLS:C0000039',
  'title': '1,2-Dipalmitoylphosphatidylcholine',
  'type': 'T109',
  'description': '1,2-Dipalmitoylphosphatidylcholine ( T109 : 1,2-Dihexadecyl-sn-Glycerophosphocholine ; 1,2-Dipalmitoyl-Glycerophosphocholine ; Dipalmitoylphosphatidylcholine ; Dipalmitoylglycerophosphocholine ; Dipalmitoyllecithin ; Dipalmitoyl Phosphatidylcholine ; 1,2 Dipalmitoylphosphatidylcholine ; 1,2 Dihexadecyl sn Glycerophosphocholine ; 1,2 Dipalmitoyl Glycerophosphocholine ; Phosphatidylcholine, Dipalmitoyl ; 3,5,9-Trioxa-4-phosphapentacosan-1-aminium, 4-hydroxy-N,N,N-trimethyl-10-oxo-7-((1-oxohexadecyl)oxy)-, inner salt, 4-oxide ; Dipalmitoylphosphatidylcholine (substance) ; DPPC ; DIPALMITOYLPHOSPHATIDYLCHOLINE 0102 ; 1,2-Dipalmitoylphosphatidylcholine [Chemical/Ingredient] ) [ Synthetic phospholipid used in liposomes and

In [6]:
import pickle
import ujson
import sys
import os
import csv
# utilities functions for medic ontology

def load_medic(ontology_dir):
    "This is needed because it's neither obo nor umls, it's a sort of preprocessing"

    key_dict = [
        "DiseaseName",
        "DiseaseID",
        "AltDiseaseIDs",
        "Definition",
        "ParentIDs",
        "TreeNumbers",
        "ParentTreeNumbers",
        "Synonyms",
        "SlimMappings",
    ]
    # Open the TSV file
    with open(ontology_dir, newline="") as tsvfile:
        # Create a CSV reader specifying the delimiter as a tab character
        reader = csv.reader(tsvfile, delimiter="\t")

        # Initialize a counter
        counter = 0

        ontology = []
        # Iterate over the rows in the file
        for row in reader:
            dict = {}
            # Print the current row
            if counter > 28:
                for i, elements in enumerate(row):
                    dict[key_dict[i]] = elements

                ontology.append(dict)
            # Increment the counter
            counter += 1

    return ontology

def medic_get_canonical_name(entities):
    """
    Get name of entities in the ontology
    data: list of dict
    """
    canonical_names = {
        entity["DiseaseID"]: entity["DiseaseName"] for entity in entities
    }
    return canonical_names


def medic_get_aliases(entities):
    """
    Get aliases of entities in the ontology
    data: list of dict
    """
    aliases = {entity["DiseaseID"]: entity["Synonyms"] for entity in entities}
    return aliases


def medic_get_definition(entities):
    """
    Get definition of entities in the ontology
    data: list of dict
    """
    definitions_dict = {
        entity["DiseaseID"]: entity["Definition"]
        for entity in entities
        if entity["Definition"] is not None
    }
    return definitions_dict


def medic_get_types(entities):
    """
    Get type of entities in the ontology
    data: list of dict
    """
    # Extract tuples of CUI and types from the Data
    types = {entity["DiseaseID"]: entity["SlimMappings"] for entity in entities}
    return types

# Function for retrieving the title + description of entities in medic_ontology
def process_medic_ontology(
    ontology,
    data_path,
    ontology_dir,
):
    """
    This function prepares the entity data : dictionary.pickle

    Parameters
    ----------
    - ontology : str (only umls for now)
    Ontology associated with the dataset
    - data_path : str
    Path where to load and save dictionary.pickle
    - ontology_dir : str
    Path to medic data
    """

    entities = load_medic(ontology_dir)

    # Get canonical name of entities in the ontology
    cui2name = medic_get_canonical_name(entities)
    # Get aliases of entities in the ontology
    cui2alias = medic_get_aliases(entities)
    # Get definition of entities in the ontology
    cui2definition = medic_get_definition(entities)
    # Get types of entities in the ontology
    cui2tui = medic_get_types(entities)

    # Check if the directory exists, and create it if it does not
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    ontology_entities = []
    for cui, name in tqdm(cui2name.items()):
        d = {}
        ent_type = cui2tui[cui]
        d["type"] = ent_type
        # other_aliases = [x for x in cui2alias[cui] if x != name]
        # joined_aliases = ' ; '.join(other_aliases)
        d["cui"] = f"{cui}"
        d["title"] = name
        if cui2definition[cui] != "":
            definition = cui2definition[cui]
        else:
            definition = None

        if cui2alias[cui] is not None:
            if definition is not None:
                d["description"] = (
                    f"{name} ( {ent_type} : {cui2alias[cui]} ) [ {definition} ]"
                )
            else:
                d["description"] = f"{name} ( {ent_type} : {cui2alias[cui]} )"
        else:
            if definition is not None:
                d["description"] = f"{name} ( {ent_type} ) [ {definition} ]"
            else:
                d["description"] = f"{name} ( {ent_type} )"

        ontology_entities.append(d)

    pickle.dump(
        ontology_entities, open(os.path.join(data_path, "dictionary.pickle"), "wb")
    )
    entities = pickle.load(open(os.path.join(data_path, "dictionary.pickle"), "rb"))
    return entities

# Function process_mention_data

In [25]:

def process_mention_dataset(ontology,
                            dataset,
                            model,
                            mention_id: Optional[bool] = False,
                            context_doc_id: Optional[bool] = False,
                            label: Optional[bool] = False
                            ): 
    '''
    This function prepares the mentions data :  Creates the train.jsonl, valid.jsonl, test.jsonl
    Each .jsonl contains data in the following format : 
    {'mention': mention, 
    'mention_id': ID of the mention, (optional)
    'context_left': context before mention,
    'context_right': context after mention, 
    'context_doc_id': ID of the doc, (optional)
    'type': type
    'label_id': label_id,
    'label': entity description, (optional)
    'label_title': entity title
    
    Parameters 
    ----------
    - dataset : str
    Name of the dataset
    - ontology : str (only umls for now)
    Ontology associated with the dataset
    '''
    data = conhelps.for_config_name(f'{dataset}_bigbio_kb').load_dataset()
    exclude = CUIS_TO_EXCLUDE[dataset]
    remap = CUIS_TO_REMAP[dataset]

    pickle_src_path = f'../data/{model}/{dataset}'
    entity_dictionary_pkl_path = os.path.join(pickle_src_path, 'dictionary.pickle')
    
    if os.path.isfile(entity_dictionary_pkl_path): 
        print("Loading stored processed entity dictionary...")
        with open(entity_dictionary_pkl_path, 'rb') as read_handle:
            entities = pickle.load(read_handle)
    # Not complete at the time I did that (it takes several process_ontology functions into account : umls / obo / mesh etc..)
    else :
        entities = process_medic_ontology(dataset, ontology)

    entity_dictionary = {d['cui']:d for d in tqdm(entities)} #CC1

    if dataset == 'ncbi_disease': #CC2
        # Need to redo this since we have multiple synonymous CUIs for ncbi_disease
        entity_dictionary = {cui:d for d in tqdm(entities) for cui in d['cui']}
        cui_synsets = {}
        for subdict in tqdm(entities): 
            for cui in subdict['cui']:
                if cui in subdict:
                    print(cui, cui_synsets[cui], subdict['cui'])
                cui_synsets[cui] = subdict['cui'] 

        with open(f'../data/{model}/{dataset}/cui_synsets.json', 'w') as f:
            f.write(ujson.dumps(cui_synsets, indent=2))

    if dataset in VALIDATION_DOCUMENT_IDS:
        validation_pmids = VALIDATION_DOCUMENT_IDS[dataset]
    else:
        print("ERROR!!!")
        
    # Convert BigBio dataset to pandas DataFrame
    df = dataset_to_df(data, entity_remapping_dict=remap, cuis_to_exclude=exclude, val_split_ids=validation_pmids)
    # Return dictionary of documents in BigBio dataset
    docs = dataset_to_documents(data)
    label_len = df['db_ids'].map(lambda x: len(x)).max()
    print("Max labels on one doc:", label_len)

    for split in df.split.unique():
        print(split)

        ents_in_split = []
        for d in tqdm(df.query("split == @split").to_dict(orient='records')):
            abbrev_resolved = False
            offsets = d['offsets']
            doc_id = d['document_id']
            doc = docs[doc_id]
            mention = d['text']
            
            # Get offsets and context
            start = offsets[0][0] # start on the mention
            end = offsets[-1][-1] # end of the mention
            before_context = doc[:start] # left context
            after_context = doc[end:] # right context
            
            
            # ArboEL can't handle multi-labels, so we randomly choose one.
            if len(d['db_ids']) == 1:
                label_id = d['db_ids'][0]

            # ncbi_disease is a special case that requires extra care
            elif dataset == 'ncbi_disease':
                labels = []
                used_cuis = set([])
                choosable_ids = []
                for db_id in d['db_ids']:
                    if db_id in used_cuis:
                        continue
                    else:
                        used_cuis.update(set(entity_dictionary[db_id]['cuis']))
                    choosable_ids.append(db_id)

                label_id = np.random.choice(choosable_ids)
            
            else:
                label_id = np.random.choice(d['db_ids'])

            # Check if we missed something
            if label_id not in entity_dictionary:
                print(label_id)
                continue
            

            
            output = [{
                'mention': mention, 
                'context_left': before_context,
                'context_right': after_context, 
                'type': d['type'][0],
                'label_id': label_id,
                'label_title': entity_dictionary[label_id]['title'],
            }]
            
            if mention_id:
                output[0]['mention_id'] = d.get('mention_id', None)
        
            if context_doc_id:
                output[0]['context_doc_id'] = d.get('context_doc_id', None)
                
            if label:
                output[0]["label"] = d.get(
                    entity_dictionary[label_id]["description"], None
                )

            ents_in_split.extend(output)

        split_name = split
        if split =='validation':
            split_name = 'valid'
        with open(f'../data/{model}/{dataset}/{split_name}.jsonl', 'w') as f:
            f.write('\n'.join([ujson.dumps(x) for x in ents_in_split]))
            

## Test if the function works

Breaks the functions into several blocks and see if each block works

In [34]:
# model = 'arboel'
# dataset = 'medmentions_st21pv'
# entities = pickle.load(open(f'/home2/cye73/data/{model}/{dataset}/dictionary.pickle', 'rb'))
# # entity_dictionary = {d['cui']:d for d in tqdm(entities)}

In [47]:
model = 'arboel'
dataset = 'ncbi_disease'
entities = pickle.load(open(f'/home2/cye73/data/{model}/{dataset}/dictionary.pickle', 'rb'))
entity_dictionary = {d['cui']:d for d in tqdm(entities)}

100%|██████████| 13189/13189 [00:00<00:00, 2455770.02it/s]


In [48]:
entity_dictionary

{'MESH:C538288': {'type': 'Congenital abnormality|Genetic disease (inborn)|Pathology (process)',
  'cui': 'MESH:C538288',
  'title': '10p Deletion Syndrome (Partial)',
  'description': '10p Deletion Syndrome (Partial) ( Congenital abnormality|Genetic disease (inborn)|Pathology (process) : Chromosome 10, 10p- Partial|Chromosome 10, monosomy 10p|Chromosome 10, Partial Deletion (short arm)|Monosomy 10p )'},
 'MESH:C535484': {'type': 'Congenital abnormality|Genetic disease (inborn)|Pathology (process)',
  'cui': 'MESH:C535484',
  'title': '13q deletion syndrome',
  'description': "13q deletion syndrome ( Congenital abnormality|Genetic disease (inborn)|Pathology (process) : Chromosome 13q deletion|Chromosome 13q deletion syndrome|Chromosome 13q monosomy|Chromosome 13q syndrome|Deletion 13q|Deletion 13q syndrome|Monosomy 13q|Monosomy 13q syndrome|Orbeli's syndrome|Orbeli syndrome )"},
 'MESH:C579849': {'type': 'Congenital abnormality|Genetic disease (inborn)|Mental disorder|Nervous system di

In [49]:
mention_id = True
context_doc_id = True
label = True

In [50]:
data = conhelps.for_config_name(f'{dataset}_bigbio_kb').load_dataset()
exclude = CUIS_TO_EXCLUDE[dataset]
remap = CUIS_TO_REMAP[dataset]

/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/ncbi_disease
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [51]:
if dataset in VALIDATION_DOCUMENT_IDS:
    validation_pmids = VALIDATION_DOCUMENT_IDS[dataset]
else:
    print("ERROR!!!")

df = dataset_to_df(data, entity_remapping_dict=remap, cuis_to_exclude=exclude, val_split_ids=validation_pmids)
docs = dataset_to_documents(data)
label_len = df['db_ids'].map(lambda x: len(x)).max()
print("Max labels on one doc:", label_len)

Max labels on one doc: 5


In [56]:
# # print("\n\ndata :\n\n", data)
# print("\n\ndf :\n\n")
# display(df)
# print("\n\ndocs :\n\n")
# display(docs)

In [75]:
for split in df.split.unique():
    print(split)

    ents_in_split = []
    for d in tqdm(df.query("split == @split").to_dict(orient='records')):
        abbrev_resolved = False
        offsets = d['offsets']
        doc_id = d['document_id']
        doc = docs[doc_id]
        mention = d['text']
        
        # Get offsets and context
        start = offsets[0][0] # start on the mention
        end = offsets[-1][-1] # end of the mention
        before_context = doc[:start] # left context
        after_context = doc[end:] # right context
        
        
        # ArboEL can't handle multi-labels, so we randomly choose one.
        if len(d['db_ids']) == 1:
            label_id = d['db_ids'][0]

        # ncbi_disease is a special case that requires extra care
        # elif dataset == 'ncbi_disease':
        #     labels = []
        #     used_cuis = set([])
        #     choosable_ids = []
        #     for db_id in d['db_ids']:
        #         if db_id in used_cuis:
        #             continue
        #         else:
        #             used_cuis.update(set(entity_dictionary[db_id]['cuis']))
        #         choosable_ids.append(db_id)

        #     label_id = np.random.choice(choosable_ids)
        
        else:
            label_id = np.random.choice(d['db_ids'])

        # Check if we missed something
        if label_id not in entity_dictionary:
            # print(label_id)
            continue
        

        
        output = {
            'mention': mention, 
            'context_left': before_context,
            'context_right': after_context, 
            'type': d['type'][0],
            'label_id': label_id,
            'label_title': entity_dictionary[label_id]['title'],
            'label' : entity_dictionary[label_id]['description']
        }
        
        if mention_id:
            output['mention_id'] = d.get('mention_id', None)
    
        if context_doc_id:
            output['context_doc_id'] = d.get('context_doc_id', None)

        ents_in_split.append(output)

train


100%|██████████| 5065/5065 [00:00<00:00, 254747.40it/s]


validation


100%|██████████| 780/780 [00:00<00:00, 263050.34it/s]


test


100%|██████████| 960/960 [00:00<00:00, 301116.65it/s]


In [76]:
ents_in_split[0]

{'mention': 'ataxia-telangiectasia',
 'context_left': 'Clustering of missense mutations in the ',
 'context_right': ' gene in a sporadic T-cell leukaemia.\nAtaxia-telangiectasia (A-T) is a recessive multi-system disorder caused by mutations in the ATM gene at 11q22-q23 (ref. 3). The risk of cancer, especially lymphoid neoplasias, is substantially elevated in A-T patients and has long been associated with chromosomal instability. By analysing tumour DNA from patients with sporadic T-cell prolymphocytic leukaemia (T-PLL), a rare clonal malignancy with similarities to a mature T-cell leukaemia seen in A-T, we demonstrate a high frequency of ATM mutations in T-PLL. In marked contrast to the ATM mutation pattern in A-T, the most frequent nucleotide changes in this leukaemia were missense mutations. These clustered in the region corresponding to the kinase domain, which is highly conserved in ATM-related proteins in mouse, yeast and Drosophila. The resulting amino-acid substitutions are pred

## load medmentions_st21pv

In [25]:
model = 'arboel'
dataset = "medmentions_st21pv"
ontology = 'UMLS'

process_mention_dataset(ontology,
                        dataset,
                        model,
                        mention_id = True,
                        context_doc_id = True,
                        label = True)

/nethome/cye73/conda_envs/arboel_2/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for bigbio/medmentions contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/medmentions
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


NameError: name 'process_ontology' is not defined